In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import random
import plotly.io as pio
import missingno as msno

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from scipy.stats import norm

cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).

# Read datasets and join on date

In [ ]:
weather_df = pd.read_csv('weather_cleaned.csv')
call_df = pd.read_csv('311_filtered.csv')

call_df.columns = call_df.columns.str.replace(' ', '_')
call_df.columns = call_df.columns.str.lower()

# keep only date of fatetime
weather_df['Date'] = pd.to_datetime(weather_df['Date']).dt.date
call_df['created_date'] = pd.to_datetime(call_df['created_date'])
call_df['created_timestamp'] = call_df['created_date']
call_df['created_date'] = call_df['created_date'].dt.date

In [ ]:
call_df.columns

In [ ]:
weather_df.columns

In [ ]:
weather_df['WBAN'].unique()

In [ ]:
# for i in weather_df['WBAN'].unique():
#     print(len(weather_df[weather_df['WBAN']==i]))

In [ ]:
weather_df[weather_df['WBAN']==465]

In [ ]:
# aggregate weather data by date and take average
weather_bydate_df = weather_df.iloc[:,4:].groupby('Date').mean()

# left join on date
combined_df = call_df.merge(weather_bydate_df, left_on='created_date', right_on='Date', how='left')

combined_df.head(5)

# Modeling
## Build time series train test split

In [ ]:
# X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
# y = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

# tscv = TimeSeriesSplit()
# print(tscv)
# TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     print ("X_train", X_train, "X_test", X_test)
#     y_train, y_test = y[train_index], y[test_index]

In [ ]:
def multivariate_regression_analysis(X_train, y_train, X_test, y_test, method, alpha, L1_wt, regularized=False):
    '''
    Takes in a dataframe (X, y)
    X - independent features
    y - target, number of daily 311 calls
    Returns regression summary
    '''
    
    # scale data
    col_names = X_train.columns
    transformer = RobustScaler().fit(X_train)
    X_train_transformed = pd.DataFrame(transformer.transform(X_train), columns=col_names)
#     print ('X train transformed shape', X_train_transformed.shape)
#     print ('y train  shape', y_train.values.reshape(-1,1).shape)
#     print("X_train_trans", X_train_transformed[0])
    X_test_transformed = pd.DataFrame(transformer.transform(X_test), columns=col_names)
#     print ('X test transformed shape', X_test_transformed.shape)
    
    #fitting the model
    
    # unregularized
    if not regularized:
#         print ('Running unregularized model...')
        model= sm.OLS(y_train.values.reshape(-1,1), X_train_transformed).fit()         
        #summary of the model
        summary= model.summary()

        pred = model.predict(X_test_transformed)
        
        mse_result = mean_squared_error(y_test, pred)
        mae_result = mean_absolute_error(y_test, pred)
        
    # regularized
    else:
#         print ('Running regularized model...')
        model= sm.OLS(y_train.values.reshape(-1,1), X_train_transformed).fit_regularized(method=method, alpha=alpha,\
                                                                                         L1_wt=L1_wt, refit=True)      
        #summary of the model
        summary= model.summary()

        pred = model.predict(X_test_transformed)
        mse_result = mean_squared_error(y_test, pred)
        mae_result = mean_absolute_error(y_test, pred)
        
    return summary, mse_result, mae_result, pred
   

def timeseries_train_test_split(X, y, date, n_splits=5, max_train_size=60, test_size = 7, regularized=False, method='elastic_net', alpha=1.0, L1_wt=0.5):
    if not regularized:
        print ('Running unregularized model...')
    else:
        print ('Running regularized model...')
        
    performance_ls = []
    summary_ls = []
    pred_ls = []
    
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size, test_size = test_size)
    for train_index, test_index in tscv.split(X):
#         print("TRAIN:", len(train_index), "TEST:", len(test_index))
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        date_index = date[test_index]
        
        summary, mse, mae, pred = multivariate_regression_analysis(X_train, y_train, X_test, y_test,\
                                                             regularized=regularized, method=method, alpha=alpha, L1_wt=L1_wt)
#         print(type(date_index))
#         print(type(pred))
        
        performance_ls.append([mse, mae])
        summary_ls.append(summary)
        
        pred_dict = {'date':date_index.values, 'pred':pred.values}
        pred_ls.append(pd.DataFrame(pred_dict))
        
    return performance_ls, summary_ls, pred_ls



In [ ]:
def results_table(ls):
    res = pd.DataFrame(np.array(performance_ls), columns=['MSE', 'MAE'])
    res['RMSE'] = np.sqrt(res['MSE'])
    
    return res

def plot_results(df, col, title):
    fig = px.line(df, x=df.index, y=col)
    fig.update_layout(
        title={
            'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            yaxis_range = [0,10_000])
    fig.show()

def pred_v_actual_plot(df, pred_df):

    # visualize the time range and call volume
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['created_date'], y=df['num_calls'], name='Num of Calls',
                            ))
    fig.add_trace(go.Scatter(x=pred_df['date'], y=pred_df['pred'], name='Predicted Num of Calls',
                            ))


    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        title={
            'text': "Predicted v.s. Actual 311 Calls",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.show()

def calc_avg_error(df):
    print ('Mean absolute error (average): ', df['MAE'].mean())
    print ('Root mean squared error (average): ', df['RMSE'].mean())

## Build a simple regression model to predict total daily call volumnes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import statsmodels.api as sm

In [ ]:
# combined_df['incident_zip'].head()

In [ ]:
combined_df.columns

### Remove certain features 
Based on previous EDA on the two datasets, we can drop based on correlation (multicolinearity), percentage of missing values, and domain knowledge.

In [ ]:
# dropped: 'agency_name', 'Gust', 'SnowDepth', 'resolution_description', 'resolution_action_updated_date'
# 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2', 'closed_date', 'created_timestamp'
# 'hour', 'month', 'day', 'complaint_type', 'park_facility_name', 'city', 'status', 'latitude', 'longitude', 'location'

col_ls = ['unique_key', 'created_date', 'agency',
       'location_type', 'incident_zip',  'address_type',
       'community_board', 'borough', 'open_data_channel_type', 'park_borough',
       'complaint_type_original', 
       'day_of_week', 'Latitude',
       'Longitude', 'MeanTemp',
       'Percipitation', 'WindSpeed', 'MaxSustainedWind', 'Rain',
       'SnowIce', 'Month', 'Day']

filtered_df = combined_df[col_ls]


### Simple regression with weather variables only
Using exog variables only and not consider the autocorrelation of call volumes by day

In [ ]:
filtered_df.head()

In [ ]:
regress_df = filtered_df.groupby('created_date').agg({'unique_key':'size', 'MeanTemp':'mean','Percipitation':'mean', 'WindSpeed':'mean', 'MaxSustainedWind':'mean', 'Rain':'mean', 'SnowIce':'mean', 'Month':'mean', 'Day':'mean', 'day_of_week':'mean', 'Latitude':'mean', 'Longitude':'mean'})\
        .rename(columns={'unique_key':'num_calls','MeanTemp':'mean_temp', 'Percipitation':'mean_precip', 'WindSpeed':'mean_ws', 'MaxSustainedWind':'mean_max_ws', 'Rain':'rain', 'SnowIce':'snowice', 'Month':'month', 'Day':'day', 'day_of_week':'day_of_wk', 'Latitude':'lat', 'Longitude':'lon'}) \
        .reset_index()

regress_df.dropna(inplace=True)
print(regress_df.shape)

# print (regress_df.shape)
regress_df.head()

In [ ]:
regress_df.corr()

In [ ]:
regress_df['num_calls'].hist()

In [ ]:
fig = make_subplots(rows=4, cols=3, shared_yaxes=True)

fig.add_trace(go.Scatter(x=regress_df['mean_temp'], y=regress_df['num_calls'], name='mean temp',mode = 'markers'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=regress_df['mean_precip'], y=regress_df['num_calls'], name='mean precipitation',mode = 'markers'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=regress_df['mean_ws'], y=regress_df['num_calls'], name='mean wind speed',mode = 'markers'),
              row=1, col=3)

fig.add_trace(go.Scatter(x=regress_df['mean_max_ws'], y=regress_df['num_calls'], name='max wind speed',mode = 'markers'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=regress_df['rain'], y=regress_df['num_calls'], name='rain',mode = 'markers'),
              row=2, col=2)

fig.add_trace(go.Scatter(x=regress_df['snowice'], y=regress_df['num_calls'], name='snpw/ice',mode = 'markers'),
              row=2, col=3)

fig.add_trace(go.Scatter(x=regress_df['year'], y=regress_df['num_calls'], name='year',mode = 'markers'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=regress_df['month'], y=regress_df['num_calls'], name='month',mode = 'markers'),
              row=3, col=2)

fig.add_trace(go.Scatter(x=regress_df['day'], y=regress_df['num_calls'], name='day',mode = 'markers'),
              row=3, col=3)

fig.add_trace(go.Scatter(x=regress_df['day_of_wk'], y=regress_df['num_calls'], name='day of week',mode = 'markers'),
              row=4, col=1)

fig.add_trace(go.Scatter(x=regress_df['lat'], y=regress_df['num_calls'], name='latitude',mode = 'markers'),
              row=4, col=2)

fig.add_trace(go.Scatter(x=regress_df['lon'], y=regress_df['num_calls'], name='longitude',mode = 'markers'),
              row=4, col=3)

fig.update_layout(height=600, width=600,
                  title_text="Multiple Subplots with Call Volumes")
fig.show()

### Calls and temperature

In [ ]:
# fig = px.scatter(regress_df, x='created_date', y="num_calls", trendline="ols")
# fig.show()


# visualize the time range and call volume
fig = go.Figure()

fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['num_calls'], name='Num of Calls',
                        yaxis='y1'
                        ))
fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['mean_temp'], name='Avg Temperature',
                        yaxis='y2'
                        ))
# fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['max_temp'], name='Max Temperature',
#                         yaxis='y2'
#                         ))


fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Daily 311 Calls",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        yaxis=dict(title='Calls'),
                       yaxis2=dict(title='Temperature',
                                   overlaying='y',
                                   side='right'))
fig.show()

### Calls and wind speed

In [ ]:
# fig = px.scatter(regress_df, x='created_date', y="num_calls", trendline="ols")
# fig.show()

regress_df.dropna(inplace=True)
print(regress_df.shape)

# visualize the time range and call volume
fig = go.Figure()

fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['num_calls'], name='Num of Calls',
                        yaxis='y1'
                        ))
fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['mean_ws'], name='Avg Wind Speed',
                        yaxis='y2'
                        ))
fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['mean_max_ws'], name='Max Wind Speed',
                        yaxis='y2'
                        ))


fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Daily 311 Calls",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        yaxis=dict(title='Calls'),
                       yaxis2=dict(title='Wind Speed',
                                   overlaying='y',
                                   side='right'))
fig.show()

### Calls and precipitation

In [ ]:
# fig = px.scatter(regress_df, x='created_date', y="num_calls", trendline="ols")
# fig.show()

regress_df.dropna(inplace=True)
print(regress_df.shape)

# visualize the time range and call volume
fig = go.Figure()

fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['num_calls'], name='Num of Calls',
                        yaxis='y1'
                        ))
fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['rain'], name='Rain',
                        yaxis='y2'
                        ))
fig.add_trace(go.Scatter(x=regress_df['created_date'], y=regress_df['snowice'], name='Snow/Ice',
                        yaxis='y2'
                        ))


fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Daily 311 Calls",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        yaxis=dict(title='Calls'),
                       yaxis2=dict(title='Precipitation',
                                   overlaying='y',
                                   side='right'))
fig.show()

### 90-Day look back window, unregularized

In [ ]:
n_splits = (len(regress_df)-90)//7

performance_ls, summary_ls, pred_ls = timeseries_train_test_split(regress_df.iloc[:,2:], regress_df.num_calls, regress_df.created_date,\
                                                         n_splits=n_splits, max_train_size=90, test_size = 7)

res = results_table(performance_ls)
display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, 90-Day Window')

In [ ]:
pred_v_actual_plot(regress_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

### Expanding window, unregularized

In [ ]:
performance_ls, summary_ls, pred_ls = timeseries_train_test_split(regress_df.iloc[:,2:], regress_df.num_calls, date=regress_df.created_date,\
                                                         n_splits=n_splits, max_train_size=None, test_size = 7)

res = results_table(performance_ls)
display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window')

In [ ]:
pred_v_actual_plot(regress_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

### Expanding window, regularized

In [ ]:
performance_ls, summary_ls, pred_ls = timeseries_train_test_split(regress_df.iloc[:,2:], regress_df.num_calls, date=regress_df.created_date,\
                                                         n_splits=n_splits, max_train_size=None, test_size = 7, regularized=True, alpha=100)

res = results_table(performance_ls)
display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window')

In [ ]:
pred_v_actual_plot(regress_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

We can see that the following variables are satistically significant:
- Mean temperature
- Mean precipitation
- Mean wind speed
- Mean max wind speed
- Rain (binary)
- Snow/ice (binary)
- Month
- Day of week

Whereas 'Precipitation' is related to 'Rain' and 'SnowIce' combined, removing them leads to a material degradation in performance, and the correlation between 'Precipitation', 'Rain' and 'SnowIce' is weak.

### Featurize total call volumes

In [ ]:
def featurize_time_series(df_in):
    df = df_in.copy()
    # create lag features
    df['lag_7'] = df['num_calls'].shift(7)
    df['lag_7'].fillna(method='bfill', inplace=True)
    df['lag_8'] = df['num_calls'].shift(8)
    df['lag_8'].fillna(method='bfill', inplace=True)
    df['lag_9'] = df['num_calls'].shift(9)
    df['lag_9'].fillna(method='bfill', inplace=True)
    df['lag_14'] = df['num_calls'].shift(14)
    df['lag_14'].fillna(method='bfill', inplace=True)

    # trailing average
    df['moving_avg7'] = df['num_calls'].shift(7).rolling(window=7, min_periods=1).mean()
    df['moving_avg7'].fillna(method='bfill', inplace=True)
    df['moving_avg30'] = df['num_calls'].shift(7).rolling(window=30, min_periods=1).mean()
    df['moving_avg30'].fillna(method='bfill', inplace=True)
    
    # cumulative moving average
    df['cm_avg'] = df['num_calls'].shift(7).expanding(min_periods=1).mean()
    df['cm_avg'].fillna(method='bfill', inplace=True)
    
    # exp moving average
    df['exp_avg'] = df['num_calls'].shift(7).ewm(com=0.5, min_periods=1).mean()
    df['exp_avg'].fillna(method='bfill', inplace=True)
    
    return df

In [ ]:
regress_featurized_df = featurize_time_series(regress_df)

In [ ]:
regress_featurized_df.head()

### Time series with weather variables, unregularized

In [ ]:
print(regress_featurized_df.shape)

performance_ls, summary_ls, pred_ls = timeseries_train_test_split(regress_featurized_df.iloc[:,2:], regress_featurized_df.num_calls, date=regress_featurized_df.created_date,\
                                                         n_splits=n_splits, max_train_size=None, test_size = 7)


res = results_table(performance_ls)
display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window w/ Historical Call Features')

In [ ]:
pred_v_actual_plot(regress_featurized_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

### Time series with weather variables, regularized

In [ ]:
performance_ls, summary_ls, pred_ls = timeseries_train_test_split(regress_featurized_df.iloc[:,2:], regress_featurized_df.num_calls, date=regress_featurized_df.created_date,\
                                                         n_splits=n_splits, max_train_size=None, test_size = 7, regularized=True, alpha=100)

res = results_table(performance_ls)
display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window w/ Historical Call Features')

In [ ]:
pred_v_actual_plot(regress_featurized_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

## Build a tree-based model
### Random forest with weather and autoregressive features

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def random_forest_analysis(X_train, y_train, X_test, y_test, n_estimators, max_depth, min_samples_split):
    '''
    Takes in a dataframe (X, y)
    X - independent features
    y - target, number of daily 311 calls
    Returns regression summary
    '''
    
#     # scale data
#     col_names = X_train.columns
#     transformer = RobustScaler().fit(X_train)
#     X_train_transformed = pd.DataFrame(transformer.transform(X_train), columns=col_names)

#     X_test_transformed = pd.DataFrame(transformer.transform(X_test), columns=col_names)
    
    #fitting the model
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,\
                                  min_samples_split=min_samples_split, n_jobs=-1)
    model = model.fit(X_train, y_train)

    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(X_train.columns, model.feature_importances_):
        feats[feature] = importance #add the name/value pair 
    summary = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    pred = model.predict(X_test)

    mse_result = mean_squared_error(y_test, pred)
    mae_result = mean_absolute_error(y_test, pred)
        
        
    return summary, mse_result, mae_result, pred
   

def timeseries_train_test_split_forest(X, y, date, n_estimators, max_depth, min_samples_split,\
                                n_splits=5, max_train_size=60, test_size = 7):

    performance_ls = []
    summary_ls = []
    pred_ls = []
    
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size, test_size = test_size)
    for train_index, test_index in tscv.split(X):
#         print("TRAIN:", len(train_index), "TEST:", len(test_index))
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        date_index = date[test_index]
        
        summary, mse, mae, pred = random_forest_analysis(X_train, y_train, X_test, y_test,\
                                                        n_estimators, max_depth, min_samples_split)
#         print(type(date_index))
#         print(type(pred))
        
        performance_ls.append([mse, mae])
        summary_ls.append(summary)
        
        pred_dict = {'date':date_index.values, 'pred':pred}
        pred_ls.append(pd.DataFrame(pred_dict))
        
    return performance_ls, summary_ls, pred_ls

In [ ]:
performance_ls, summary_ls, pred_ls = timeseries_train_test_split_forest(regress_featurized_df.iloc[:,2:], regress_featurized_df.num_calls, date=regress_featurized_df.created_date, \
                                                                  n_estimators=120, max_depth=4, min_samples_split=2, n_splits=n_splits, max_train_size=None, test_size = 7)

res = results_table(performance_ls)
# display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window w/ Historical Call Features')

In [ ]:
plot = summary_ls[-1].sort_values(by='Gini-importance')
fig = px.bar(plot, x=plot.index, y='Gini-importance')
fig.show()

In [ ]:
pred_v_actual_plot(regress_featurized_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

### Random forest with autoregressive features

In [ ]:
regress_featurized_df.columns

In [ ]:
# n_estimators=[100, 150, 200]
# max_depth=[4,6,8]
# min_samples_split=[2,4,6,8]


# param_grid = dict('n_estimators':n_estimators, 'max_depth':max_depth, 'min_sample_split':min_samples_split)

performance_ls, summary_ls, pred_ls = timeseries_train_test_split_forest(regress_featurized_df.iloc[:,8:], regress_featurized_df.num_calls, date=regress_featurized_df.created_date, \
                                                                  n_estimators=120, max_depth=4, min_samples_split=2, n_splits=n_splits, max_train_size=None, test_size = 7)

res = results_table(performance_ls)
# display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window w/ Historical Call Features')

In [ ]:
plot = summary_ls[-1].sort_values(by='Gini-importance')
fig = px.bar(plot, x=plot.index, y='Gini-importance')
fig.show()

In [ ]:
pred_v_actual_plot(regress_featurized_df, pd.concat(pred_ls).reset_index(drop=True))
calc_avg_error(res)

## Build an autoregressive time series model

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

result = adfuller(regress_df.num_calls.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(regress_df.num_calls); axes[0, 0].set_title('Original Series')
axes[0,1].set(xlim=(0,30))
plot_acf(regress_df.num_calls, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(regress_df.num_calls.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(regress_df.num_calls.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(regress_df.num_calls.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(regress_df.num_calls.diff().diff().dropna(), ax=axes[2, 1])

plt.show()

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(2, 2, sharex=True)
axes[0,0].plot(regress_df.num_calls.diff()); axes[0,0].set_title('1st Differencing')
axes[0,0].set(xlim=(0,30))
plot_pacf(regress_df.num_calls.diff().dropna(), ax=axes[0, 1])

axes[1,0].plot(regress_df.num_calls.diff().diff()); axes[1,0].set_title('2nd Differencing')
axes[1,0].set(xlim=(0,30))
plot_pacf(regress_df.num_calls.diff().diff().dropna(), ax=axes[1, 1])

plt.show()

In [ ]:
regress_df.head()

In [ ]:
# specify training data
data = regress_df.num_calls
# specify additional data
other_data = regress_df.mean_temp

# define model configuration
my_order = (2, 1, 1)
my_seasonal_order = (2, 1, 1, 7)
# define model
model = sm.tsa.statespace.SARIMAX(endog=data, exog=other_data, order=my_order, seasonal_order=my_seasonal_order)

In [ ]:
model_fit = model.fit(method='bfgs')

In [ ]:
# # split into train and test sets
# X = regress_df.num_calls
# size = int(len(X) * 0.66)
# train, test = X[0:size], X[size:len(X)]
# history = [x for x in train]
# predictions = []

# # walk-forward validation
# for t in range(size, len(X)):
#     model = sm.tsa.statespace.SARIMAX(endog=data, exog=other_data, order=my_order, seasonal_order=my_seasonal_order)
#     model_fit = model.fit(method='bfgs')
#     output = model_fit.forecast()
#     yhat = output[0]
#     predictions.append(yhat)
#     obs = test[t]
#     history.append(obs)
#     print('predicted=%f, expected=%f' % (yhat, obs))
    
# # evaluate forecasts
# rmse = np.sqrt(mean_squared_error(test, predictions))
# print('Test RMSE: %.3f' % rmse)
# # plot forecasts against actual outcomes
# plt.plot(test)
# plt.plot(predictions, color='red')
# plt.show()

In [ ]:
regress_df.head()

arima_df = pd.DataFrame()
arima_df['date'] = pd.to_datetime(regress_df['created_date'])
arima_df['mean_temp'] = regress_df['mean_temp']
arima_df['num_calls'] = regress_df['num_calls']
arima_df.set_index('date', inplace=True)

arima_df.index = pd.DatetimeIndex(arima_df.index.values,
                               freq=arima_df.index.inferred_freq)

ts = arima_df['num_calls']

In [ ]:
plt.plot(ts, label='prediction seasonal')
plt.grid()
plt.xticks(rotation=90)
plt.show()

In [ ]:
result = seasonal_decompose(ts, model='additive',extrapolate_trend='freq')
result.plot()
plt.show()

In [ ]:
def check_stationarity(ts):
    dftest = adfuller(ts)
    adf = dftest[0]
    pvalue = dftest[1]
    critical_value = dftest[4]['5%']
    if (pvalue < 0.05) and (adf < critical_value):
        print('The series is stationary')
    else:
        print('The series is NOT stationary')
        
seasonal = result.seasonal
check_stationarity(seasonal)

In [ ]:
ts_train = arima_df[:'2018-03-01']['num_calls']
ts_test = arima_df['2018-03-02':]['num_calls']

model=sm.tsa.statespace.SARIMAX(endog=ts_train, exog=arima_df[:'2018-03-01'].mean_temp,order=(2, 1, 1),seasonal_order=(2,1,1,7))
results=model.fit()

In [ ]:
# summary of fit model
print(results.summary())
# line plot of residuals
residuals = pd.DataFrame(results.resid)
residuals.plot()
plt.show()
# density plot of residuals
residuals.plot(kind='kde')
plt.show()
# summary stats of residuals
print(residuals.describe())

In [ ]:
# arima_df['forecast']=results.get_forecast('2018-03-02')
out = pd.DataFrame(results.forecast(7, exog=arima_df['2018-03-02':'2018-03-08'].mean_temp))
out

In [ ]:
# visualize the time range and call volume
fig = go.Figure()

fig.add_trace(go.Scatter(x=arima_df.index, y=arima_df['num_calls'], name='Num of Calls',
                        yaxis='y1'
                        ))
fig.add_trace(go.Scatter(x=out.index, y=out['predicted_mean'], name='Prediction',
                        yaxis='y1'
                        ))


fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title={
        'text': "Daily 311 Calls",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        yaxis=dict(title='Calls'))
fig.show()

In [ ]:
def sarimax_analysis(endog_train, exog_train, endog_test, exog_test, order=(2,1,1), seasonal_order=(2,1,1,7)):
    '''
    Takes in a endogenous and exognenous variables, as well as other parameters

    Returns regression summary
    '''
    
    
    #fitting the model
    model = sm.tsa.statespace.SARIMAX(endog=endog_train, exog=exog_train,order=order,seasonal_order=seasonal_order, freq='D')
    results=model.fit(disp=False)
    pred = results.forecast(7, exog=exog_test)
#     display(pred)
    summary = results.summary()
    
    mse_result = mean_squared_error(endog_test, pred)
    mae_result = mean_absolute_error(endog_test, pred)
        
        
    return summary, mse_result, mae_result, pred
   

def timeseries_train_test_split_sarimax(endog, exog, date, order=(2,1,1), seasonal_order=(2,1,1,7),\
                                n_splits=120, max_train_size=90, test_size = 7):

    performance_ls = []
    summary_ls = []
    pred_ls = []
    cnt = 0
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size, test_size = test_size)
    for train_index, test_index in tscv.split(endog):
        cnt += 1
        if cnt % 10 == 0:
            print('Training {} split'.format(cnt))
        endog_train, endog_test = endog[train_index], endog[test_index]
        exog_train, exog_test = exog[train_index], exog[test_index]

        date_index = date[test_index]
        
        summary, mse, mae, pred = sarimax_analysis(endog_train=endog_train, exog_train=exog_train, endog_test=endog_test, exog_test=exog_test,\
                                                   order=order, seasonal_order=seasonal_order)
#         display(pred)

        pred_df = pd.DataFrame(pred)
        performance_ls.append([mse, mae])
        summary_ls.append(summary)
        
        pred_ls.append(pred_df)
        
    return performance_ls, summary_ls, pred_ls

In [ ]:
endog = arima_df['num_calls']
exog = arima_df['mean_temp']
date = arima_df.index
performance_ls, summary_ls, pred_ls = timeseries_train_test_split_sarimax(endog, exog, date, order=(1,1,1), seasonal_order=(1,1,1,7),\
                                n_splits=120, max_train_size=None, test_size = 7)

In [ ]:
res = results_table(performance_ls)
# display(summary_ls[-1])

plot_results(res, ["MAE", 'RMSE'], 'Look Foward 7 Day 311 Call Volume Performance, Expanding Window w/ Historical Call Features')

In [ ]:
res

In [ ]:
display(np.percentile(arima_df.num_calls, 15))
display(np.percentile(arima_df.num_calls, 85))

In [ ]:
display(arima_df.num_calls.min())
display(arima_df.num_calls.max())

In [ ]:
def pred_v_actual_plot(df, pred_df):

    # visualize the time range and call volume
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=df['num_calls'], name='Num of Calls',
                            ))
    fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['predicted_mean'], name='Predicted Num of Calls',
                            ))


    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        title={
            'text': "Predicted v.s. Actual 311 Calls",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.show()

pred_v_actual_plot(arima_df, pd.concat(pred_ls))
calc_avg_error(res)